In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Probit
import matplotlib.pyplot as plt
import scipy.stats as stats
import openpyxl as xl


Data Prep

Laod the rates.xlsx File and perform a stadard date fromatting
create a new variable that computes the spread betewwen the 10y DGS10 and the 3 Month DGS3MO yealds by substracting the 3 month from the 10 Year
Aggregate the daily Data to the montly frequencey, keeping the last daily observation for each month

auslesen

In [2]:
rates = pd.read_excel("rates.xlsx", engine='openpyxl')


sortieren und date to time

In [3]:
rates["observation_date"] = pd.to_datetime(rates["observation_date"])
rates = rates.sort_values("observation_date").reset_index(drop=True)

compare and adregate

In [4]:
# Compute term spread between 10Y and 3M yields
rates['term_spread_10y_3m'] = rates['DGS10'] - rates['DGS3MO']
rates_monthly = rates.set_index("observation_date").resample("ME").last().reset_index()

rates_monthly.head()

,observation_date,fed_funds_futures,credit_spread,5y5y,DGS3MO,DGS10,term_spread_10y_3m
0,2010-01-31,99.8875,1.83,2.75,0.08,3.63,3.55
1,2010-02-28,99.8725,1.84,2.43,0.13,3.61,3.48
2,2010-03-31,99.8325,1.61,2.65,0.16,3.84,3.68
3,2010-04-30,99.8000,1.55,2.79,0.16,3.69,3.53
4,2010-05-31,99.7950,2.02,2.29,0.16,3.31,3.15


What is the minimum FED Funds

In [5]:
min_price = rates["fed_funds_futures"].min()
print(min_price)

94.6625


max

In [6]:
max_spread = rates["fed_funds_futures"].max()
print(max_spread)

99.9525


neue file laseden formatten sortieren und dann aggregiert den monats price und die letzte observation behlaten


In [7]:
stock = pd.read_excel("stock.xlsx", engine='openpyxl')
stock["observation_date"] = pd.to_datetime(stock["observation_date"])
stock = stock.sort_values("observation_date").reset_index(drop=True)


Aggregieren montly

In [8]:
# Resample to month-end so dates align with monthly market data
stocks_monthly = (
    stock[["observation_date", "Close"]]
    .set_index("observation_date")
    .resample("ME")
    .last()
    .reset_index()
)
stocks_monthly.head()

,observation_date,Close
0,2010-01-31,27.125
1,2010-02-28,26.360
2,2010-03-31,27.500
3,2010-04-30,26.725
4,2010-05-31,25.700


avg

In [9]:
avg_month = stocks_monthly["Close"].mean()
print(avg_month)

47.552269736842106


Market einlesen

In [10]:
mkt = pd.read_excel("mkt.xlsx", engine='openpyxl')


Zahlen in Month umwandeln

In [11]:
mkt["month"] = pd.to_datetime(mkt["month"].astype(str),format="%Y%m")


Prozentwerte Skalieren

In [12]:
mkt['mkt_rf_dec'] = mkt['Mkt-RF'].astype(float) / 100
mkt['rf_dec'] = mkt['RF'].astype(float) / 100
mkt.head()

,month,Mkt-RF,RF,mkt_rf_dec,rf_dec
0,1926-07-01,2.89,0.22,0.0289,0.0022
1,1926-08-01,2.64,0.25,0.0264,0.0025
2,1926-09-01,0.38,0.23,0.0038,0.0023
3,1926-10-01,-3.27,0.32,-0.0327,0.0032
4,1926-11-01,2.54,0.31,0.0254,0.0031


Date rausziehen

In [13]:
from pandas._libs.tslibs.offsets import MonthEnd

mkt['observation_date'] = mkt['month'] + MonthEnd(0)
mkt.head()

,month,Mkt-RF,RF,mkt_rf_dec,rf_dec,observation_date
0,1926-07-01,2.89,0.22,0.0289,0.0022,1926-07-31
1,1926-08-01,2.64,0.25,0.0264,0.0025,1926-08-31
2,1926-09-01,0.38,0.23,0.0038,0.0023,1926-09-30
3,1926-10-01,-3.27,0.32,-0.0327,0.0032,1926-10-31
4,1926-11-01,2.54,0.31,0.0254,0.0031,1926-11-30


merge

In [14]:
merge = stocks_monthly.merge(mkt, on="observation_date", how="inner")
merge.head()

,observation_date,Close,month,Mkt-RF,RF,mkt_rf_dec,rf_dec
0,2010-01-31,27.125,2010-01-01,-3.35,0.00,-0.0335,0.0000
1,2010-02-28,26.360,2010-02-01,3.39,0.00,0.0339,0.0000
2,2010-03-31,27.500,2010-03-01,6.30,0.01,0.0630,0.0001
3,2010-04-30,26.725,2010-04-01,1.99,0.01,0.0199,0.0001
4,2010-05-31,25.700,2010-05-01,-7.90,0.01,-0.0790,0.0001


rendiete

In [15]:
merge["stock_return"] = merge["Close"] / merge["Close"].shift(1) - 1
merge.head()

,observation_date,Close,month,Mkt-RF,RF,mkt_rf_dec,rf_dec,stock_return
0,2010-01-31,27.125,2010-01-01,-3.35,0.00,-0.0335,0.0000,NaN
1,2010-02-28,26.360,2010-02-01,3.39,0.00,0.0339,0.0000,-0.028203
2,2010-03-31,27.500,2010-03-01,6.30,0.01,0.0630,0.0001,0.043247
3,2010-04-30,26.725,2010-04-01,1.99,0.01,0.0199,0.0001,-0.028182
4,2010-05-31,25.700,2010-05-01,-7.90,0.01,-0.0790,0.0001,-0.038354


excess retrun

In [16]:
merge["excess"] = merge["stock_return"] - merge["rf_dec"]
merge.head()

,observation_date,Close,month,Mkt-RF,RF,mkt_rf_dec,rf_dec,stock_return,excess
0,2010-01-31,27.125,2010-01-01,-3.35,0.00,-0.0335,0.0000,NaN,NaN
1,2010-02-28,26.360,2010-02-01,3.39,0.00,0.0339,0.0000,-0.028203,-0.028203
2,2010-03-31,27.500,2010-03-01,6.30,0.01,0.0630,0.0001,0.043247,0.043147
3,2010-04-30,26.725,2010-04-01,1.99,0.01,0.0199,0.0001,-0.028182,-0.028282
4,2010-05-31,25.700,2010-05-01,-7.90,0.01,-0.0790,0.0001,-0.038354,-0.038454


Rates

In [17]:
rates = pd.read_excel("rates.xlsx", engine="openpyxl")

# Convert to month start for alignment with mkt "month" key
rates["month"] = (
    pd.to_datetime(rates["observation_date"])
    .dt.to_period("M")
    .dt.to_timestamp()
)

rates = rates.sort_values("observation_date").copy()
rates["term_spread_10y_3m"] = rates["DGS10"] - rates["DGS3MO"]

# Monthly last observation + first-difference of macro series
rates_m = (
    rates
    .groupby("month")
    .last()
    .reset_index()
)

for col in [
    "fed_funds_futures",
    "credit_spread",
    "5y5y",
    "term_spread_10y_3m",
]:
    rates_m[f"{col}_chg"] = rates_m[col].diff()

macro_changes = rates_m[[
    "month",
    "fed_funds_futures_chg",
    "credit_spread_chg",
    "5y5y_chg",
    "term_spread_10y_3m_chg",
]]

merge = (
    merge
    .merge(macro_changes, on="month", how="inner")
    .dropna()
    .reset_index(drop=True)
)
merge.head()

,observation_date,Close,month,Mkt-RF,RF,mkt_rf_dec,rf_dec,stock_return,excess,fed_funds_futures_chg,credit_spread_chg,5y5y_chg,term_spread_10y_3m_chg
0,2010-02-28,26.360,2010-02-01,3.39,0.00,0.0339,0.0000,-0.028203,-0.028203,-0.0150,0.01,-0.32,-0.07
1,2010-03-31,27.500,2010-03-01,6.30,0.01,0.0630,0.0001,0.043247,0.043147,-0.0400,-0.23,0.22,0.20
2,2010-04-30,26.725,2010-04-01,1.99,0.01,0.0199,0.0001,-0.028182,-0.028282,-0.0325,-0.06,0.14,-0.15
3,2010-05-31,25.700,2010-05-01,-7.90,0.01,-0.0790,0.0001,-0.038354,-0.038454,-0.0050,0.47,-0.50,-0.38
4,2010-06-30,25.060,2010-06-01,-5.56,0.01,-0.0556,0.0001,-0.024903,-0.025003,0.0275,0.07,-0.19,-0.36


Observations

In [18]:
print(len(merge))

187


sharp ratio

In [19]:
sharpe_ratio = merge['excess'].mean() / merge['excess'].std()
annualized_sharpe = sharpe_ratio * np.sqrt(12)
print(annualized_sharpe)
print(sharpe_ratio)

0.38824383344463986
0.11207634087523753


OLS Regression

In [26]:
predictors = [
    "fed_funds_futures_chg",
    "credit_spread_chg",
    "5y5y_chg",
    "term_spread_10y_3m_chg",
]
max_lag = 12

X_current = merge[predictors]
lagged_blocks = [X_current]
for lag in range(1, max_lag + 1):
    lagged_block = X_current.shift(lag)
    lagged_block.columns = [f"{col}_lag{lag}" for col in predictors]
    lagged_blocks.append(lagged_block)

X_full = pd.concat(lagged_blocks, axis=1)
regression_df = pd.concat([merge[["mkt_rf_dec"]], X_full], axis=1).dropna()

Y = regression_df["mkt_rf_dec"]
X = regression_df.drop(columns="mkt_rf_dec")

X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             mkt_rf_dec   R-squared:                       0.678
Model:                            OLS   Adj. R-squared:                  0.541
Method:                 Least Squares   F-statistic:                     4.944
Date:                Sat, 10 Jan 2026   Prob (F-statistic):           2.12e-13
Time:                        09:42:35   Log-Likelihood:                 403.44
No. Observations:                 175   AIC:                            -700.9
Df Residuals:                     122   BIC:                            -533.2
Df Model:                          52                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                            0.0091      0.002      3.712      0.000       0.004       0.014
fed_funds_futures_chg            0.0996      0.031      3.201      0.002       0.038       0.161
credit_spread_chg               -0.1456      0.019     -7.857      0.000      -0.182      -0.109
5y5y_chg                         0.0688      0.024      2.923      0.004       0.022       0.115
term_spread_10y_3m_chg          -0.0418      0.012     -3.459      0.001      -0.066      -0.018
fed_funds_futures_chg_lag1      -0.0499      0.040     -1.257      0.211      -0.129       0.029
credit_spread_chg_lag1          -0.0128      0.019     -0.673      0.502      -0.050       0.025
5y5y_chg_lag1                   -0.0148      0.025     -0.603      0.548      -0.063       0.034
term_spread_10y_3m_chg_lag1      0.0041      0.012      0.335      0.739      -0.020       0.029
fed_funds_futures_chg_lag2       0.0285      0.040      0.716      0.475      -0.050       0.107
credit_spread_chg_lag2          -0.0207      0.019     -1.080      0.282      -0.059       0.017
5y5y_chg_lag2                   -0.0541      0.025     -2.172      0.032      -0.103      -0.005
term_spread_10y_3m_chg_lag2      0.0156      0.013      1.157      0.250      -0.011       0.042
fed_funds_futures_chg_lag3      -0.0566      0.040     -1.427      0.156      -0.135       0.022
credit_spread_chg_lag3           0.0016      0.020      0.080      0.937      -0.038       0.041
5y5y_chg_lag3                    0.0158      0.025      0.623      0.534      -0.034       0.066
term_spread_10y_3m_chg_lag3     -0.0257      0.013     -1.912      0.058      -0.052       0.001
fed_funds_futures_chg_lag4       0.0191      0.039      0.484      0.629      -0.059       0.097
credit_spread_chg_lag4          -0.0184      0.020     -0.928      0.355      -0.058       0.021
5y5y_chg_lag4                    0.0118      0.024      0.490      0.625      -0.036       0.059
term_spread_10y_3m_chg_lag4   3.552e-05      0.013      0.003      0.998      -0.027       0.027
fed_funds_futures_chg_lag5      -0.0652      0.040     -1.638      0.104      -0.144       0.014
credit_spread_chg_lag5           0.0271      0.019      1.427      0.156      -0.011       0.065
5y5y_chg_lag5                    0.0114      0.023      0.501      0.617      -0.034       0.056
term_spread_10y_3m_chg_lag5      0.0146      0.013      1.133      0.259      -0.011       0.040
fed_funds_futures_chg_lag6       0.0273      0.039      0.700      0.485      -0.050       0.105
credit_spread_chg_lag6          -0.0181      0.019     -0.939      0.350      -0.056       0.020
5y5y_chg_lag6                   -0.0343      0.023     -1.507      0.134      -0.079       0.011
term_spread_10y_3m_chg_lag6     -0.0048      0.013     -0.384      0.702    

USRCE Laden, formatieren

In [21]:
usrce = pd.read_excel("USREC-2.xlsx", engine='openpyxl')
usrce.head()

,month,USREC
0,1900-02-01,1
1,1900-03-01,1
2,1900-04-01,1
3,1900-05-01,1
4,1900-06-01,1


In [22]:
usrce["month"] = pd.to_datetime(usrce["month"])
usrce["month"] = usrce["month"].dt.to_period("M").dt.to_timestamp()
usrce = usrce.sort_values("month").reset_index(drop=True)

merge = merge.merge(usrce, on="month", how="inner")

merge.head()

,observation_date,Close,month,Mkt-RF,RF,mkt_rf_dec,rf_dec,stock_return,excess,fed_funds_futures_chg,credit_spread_chg,5y5y_chg,term_spread_10y_3m_chg,USREC
0,2010-02-28,26.360,2010-02-01,3.39,0.00,0.0339,0.0000,-0.028203,-0.028203,-0.0150,0.01,-0.32,-0.07,0
1,2010-03-31,27.500,2010-03-01,6.30,0.01,0.0630,0.0001,0.043247,0.043147,-0.0400,-0.23,0.22,0.20,0
2,2010-04-30,26.725,2010-04-01,1.99,0.01,0.0199,0.0001,-0.028182,-0.028282,-0.0325,-0.06,0.14,-0.15,0
3,2010-05-31,25.700,2010-05-01,-7.90,0.01,-0.0790,0.0001,-0.038354,-0.038454,-0.0050,0.47,-0.50,-0.38,0
4,2010-06-30,25.060,2010-06-01,-5.56,0.01,-0.0556,0.0001,-0.024903,-0.025003,0.0275,0.07,-0.19,-0.36,0


Ols regression x = monthly stock return
y = US recession indicator
12 lags

In [23]:
# OLS: US recession indicator on monthly stock return (current + 12 lags)
# Ensure USREC is present even if cells were run out of order

max_lag = 12

X_current = merge[["stock_return"]]
lagged_blocks = [X_current]
for lag in range(1, max_lag + 1):
    lagged_block = X_current.shift(lag)
    lagged_block.columns = [f"stock_return_lag{lag}"]
    lagged_blocks.append(lagged_block)

X_full = pd.concat(lagged_blocks, axis=1)
regression_df = pd.concat([merge[["USREC"]], X_full], axis=1).dropna()

Y = regression_df["USREC"]
X = regression_df.drop(columns="USREC")
X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  USREC   R-squared:                       0.212
Model:                            OLS   Adj. R-squared:                  0.149
Method:                 Least Squares   F-statistic:                     3.335
Date:                Sat, 10 Jan 2026   Prob (F-statistic):           0.000156
Time:                        09:41:39   Log-Likelihood:                 164.83
No. Observations:                 175   AIC:                            -301.7
Df Residuals:                     161   BIC:                            -257.3
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0173      0.010      1.733      0.085      -0.002       0.037
stock_return          -0.6285      0.180     -3.487      0.001      -0.984      -0.273
stock_return_lag1     -0.9298      0.183     -5.080      0.000      -1.291      -0.568
stock_return_lag2     -0.3308      0.188     -1.764      0.080      -0.701       0.040
stock_return_lag3      0.0612      0.188      0.326      0.745      -0.310       0.432
stock_return_lag4     -0.0639      0.186     -0.344      0.731      -0.431       0.303
stock_return_lag5     -0.1454      0.186     -0.781      0.436      -0.513       0.222
stock_return_lag6     -0.1434      0.186     -0.773      0.441      -0.510       0.223
stock_return_lag7      0.0127      0.185      0.069      0.945      -0.352       0.378
stock_return_lag8      0.2426      0.184      1.321      0.188      -0.120       0.605
stock_return_lag9      0.2493      0.186      1.341      0.182      -0.118       0.616
stock_return_lag10     0.1280      0.186      0.688      0.492      -0.239       0.495
stock_return_lag11     0.1330      0.185      0.718      0.474      -0.233       0.499
stock_return_lag12     0.2090      0.182      1.149      0.252      -0.150       0.568
==============================================================================
Omnibus:                      255.291   Durbin-Watson:                   1.041
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19578.161
Skew:                           6.376   Prob(JB):                         0.00
Kurtosis:                      53.223   Cond. No.                         41.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
#'hier

In [25]:
'''
# Simple OLS template (copy/paste and adapt)
import statsmodels.api as sm

# y = Zielvariable
# X = Regressoren (als DataFrame)
y = df["y_variable"]
X = df[["x1", "x2", "x3"]]

# Intercept hinzufuegen
X = sm.add_constant(X)

# OLS schaetzen
model = sm.OLS(y, X).fit()
print(model.summary())d
'''

'\n# Simple OLS template (copy/paste and adapt)\nimport statsmodels.api as sm\n\n# y = Zielvariable\n# X = Regressoren (als DataFrame)\ny = df["y_variable"]\nX = df[["x1", "x2", "x3"]]\n\n# Intercept hinzufuegen\nX = sm.add_constant(X)\n\n# OLS schaetzen\nmodel = sm.OLS(y, X).fit()\nprint(model.summary())d\n'